In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# ============================
# 1. Load and Preprocess Training Data
# ============================

# Load the training dataset
df = pd.read_csv("/train.csv")

# Extract pixel values and labels
pixels = df.drop(columns=["label"]).values  #784 total
labels = df["label"].values

# Convert data to tensors
pixels_tensor = torch.tensor(pixels, dtype=torch.float32) / 255.0  # Normalize to 0,1
labels_tensor = torch.tensor(labels, dtype=torch.long)

# ============================
# 2. Define the Neural Network Model
# ============================

model = nn.Sequential(
    nn.Linear(784, 128),  # Input layer (784 -> 128)
    nn.ReLU(),            # Activation function
    nn.Linear(128, 64),   # Hidden layer (128 -> 64)
    nn.ReLU(),            # Activation function
    nn.Linear(64, 10)     # Output layer (64 -> 10 classes)
)

# Define loss function (cross-entropy for classification)
loss_fn = nn.CrossEntropyLoss()

# Define optimizer (Adam optimizer with learning rate 0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ============================
# 3. Train the Model
# ============================

# Create a DataLoader to handle mini-batches (batch size = 32)
train_loader = DataLoader(
    TensorDataset(pixels_tensor, labels_tensor), batch_size=32, shuffle=True
)

epochs = 10
model.train()

for epoch in range(epochs):
    for images, targets in train_loader:
        optimizer.zero_grad()  # Reset gradients before each batch

        outputs = model(images)  # Forward pass
        loss = loss_fn(outputs, targets)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights


# ============================
# 4. Load and Preprocess Test Data
# ============================

# Load test dataset
test_df = pd.read_csv("/test.csv")
test_pixels = test_df.values

# Convert to PyTorch tensor and normalize
test_pixels_tensor = torch.tensor(test_pixels, dtype=torch.float32) / 255.0

# ============================
# 5. Make Predictions
# ============================

model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculations
    predictions = model(test_pixels_tensor)

# Get the class with the highest predicted probability
predicted_classes = torch.argmax(predictions, dim=1)

# ============================
# 6. Save Predictions to CSV
# ============================

submission_df = pd.DataFrame({
    "ImageId": range(1, len(predicted_classes) + 1),
    "Label": predicted_classes.numpy()
})

# Save the predictions to a CSV file
submission_df.to_csv("/predictions.csv", index=False)

print("Done :D")


Done :D
